# Дообучение T5 на задачу суммаризации статей

**Цель домашнего задания**: дообучить T5 на задачу суммаризации и применить для придумывания заголовков к статьям. Задеплоить модель с помощь Hugging Face Space.

### План работ:

- Найдем подходящий набор данных, содержащий текстовое содержание и заголовки статей.
- Выберем подходящую метрику для нашей задачи.
- Тонкая настройка предварительно обученной модели для генерации заголовков и сохранение контрольных точек модели на Google Диске (чтобы мы могли возобновить обучение в случае, если Colab отключит соединение).
- Загрузим модель на Hugging Face Hub, чтобы все могли ею воспользоваться.
- Создадим интерактивную демонстрацию с помощью Streamlit и разместим ее в Hugging Face Spaces.

In [4]:
!pip install datasets transformers rouge_score nltk -qqq

  Preparing metadata (setup.py) ... done


In [5]:
pip install --upgrade datasets pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 require

### Загрузка данных

Подключимся к Google диску для того, чтобы заливать туда веса.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import transformers
from datasets import load_dataset

Загрузим датасет из 190к статей с портала Medium.

In [2]:
medium_datasets = load_dataset("csv", data_files="/content/drive/MyDrive/datasets/medium_articles.csv")

Generating train split: 0 examples [00:00, ? examples/s]

Каждая строка в данных — это отдельная статья, опубликованная на Medium. Для каждой статьи у вас есть следующие колонки:

- title [string] : Название статьи.
- текст [строка] : Текстовое содержание статьи.
- url [строка] : URL-адрес, связанный со статьей.
- авторы [список строк] : Авторы статьи.
- timestamp [string] : дата и время публикации статьи.
- теги [список строк] : Список тегов, связанных со статьей.

In [3]:
datasets_train_test = medium_datasets["train"].train_test_split(test_size=3000)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=3000)

medium_datasets["train"] = datasets_train_validation["train"]
medium_datasets["validation"] = datasets_train_validation["test"]
medium_datasets["test"] = datasets_train_test["test"]

Использовать весь датасет не будем, возьмем лишь часть.

In [4]:
medium_datasets["train"] = medium_datasets["train"].shuffle().select(range(100000))
medium_datasets["validation"] = medium_datasets["validation"].shuffle().select(range(1000))
medium_datasets["test"] = medium_datasets["test"].shuffle().select(range(1000))

In [5]:
medium_datasets['train'][0]

{'title': 'Thinking, Fast and Slow by Daniel Kahneman',
 'text': 'Thinking, Fast and Slow by Daniel Kahneman Book Summary\n\nMajor New York Times bestseller\n\nWinner of the National Academy of Sciences Best Book Award in 2012\n\nSelected by the New York Times Book Review as one of the ten best books of 2011\n\nA Globe and Mail Best Books of the Year 2011 Title\n\nOne of The Economist’s 2011 Books of the Year\n\nOne of The Wall Street Journal’s Best Nonfiction Books of the Year 2011\n\n2013 Presidential Medal of Freedom Recipient\n\nKahneman’s work with Amos Tversky is the subject of Michael Lewis’s The Undoing Project: A Friendship That Changed Our Minds\n\nIn the international bestseller, Thinking, Fast and Slow, Daniel Kahneman, the renowned psychologist and winner of the Nobel Prize in Economics, takes us on a groundbreaking tour of the mind and explains the two systems that drive the way we think. System 1 is fast, intuitive, and emotional; System 2 is slower, more deliberative, a

### Предобработка данных

In [6]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Прежде чем применять токенизатор к данным, давайте отфильтруем некоторые плохие образцы (т. е. статьи, заголовок которых короче 20 символов, а текстовое содержимое короче 500 символов).

In [7]:
def filter_ds(row):
  return len(str(row['title'])) > 20 and len(str(row['text'])) > 500

medium_datasets_cleaned = medium_datasets.filter(filter_ds)

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
print('Исходный размер датасета: ', medium_datasets.shape)
print('Размер почищенного датасета: ', medium_datasets_cleaned.shape)

Исходный размер датасета:  {'train': (100000, 6), 'validation': (1000, 6), 'test': (1000, 6)}
Размер почищенного датасета:  {'train': (84608, 6), 'validation': (836, 6), 'test': (841, 6)}


**Максимальный балл - 10**

Функция `preprocess_data` должна делать следующее:

1) Извлекает «текстовый» признак из каждого образца (т. е. текстовое содержимое статьи), исправляет переносы строк в статье и удаляет строки без завершающих знаков препинания (т. е. подзаголовки).

2) Добавляет инструкцию «summarize: к каждому тексту статьи, что необходимо для точной настройки модели T5.

3) Применяет токенизатор T5 к тексту статьи, создав `model_inputs` объект. Этот объект представляет собой словарь, содержащий для каждой статьи `input_ids` и `attention_mask` массивы, содержащие идентификаторы токенов и маски внимания соответственно.

In [15]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [11]:
prefix = "summarize: "
max_input_length = 512
max_target_length = 64

def clean_text(text):
  sentences = nltk.sent_tokenize(text.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
  sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
                                 if len(sent) > 0 and
                                 sent[-1] in string.punctuation]
  text_cleaned = "\n".join(sentences_cleaned_no_titles)
  return text_cleaned

def preprocess_data(examples):
  texts_cleaned = [clean_text(text) for text in examples["text"]]
  inputs = [prefix + text for text in texts_cleaned]


  model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

  labels = tokenizer(
        examples["title"],
        max_length=max_target_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

**Максимальный балл - 10**

Функцию `preprocess_data` можно применить ко всем наборам данных с помощью `map` метода.

In [12]:
tokenized_datasets = medium_datasets_cleaned.map(preprocess_data, batched=True)

Map:   0%|          | 0/84608 [00:00<?, ? examples/s]

Map:   0%|          | 0/836 [00:00<?, ? examples/s]

Map:   0%|          | 0/841 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets.save_to_disk('/content/drive/MyDrive/datasets/tokenized_datasets_2')

Saving the dataset (0/2 shards):   0%|          | 0/84608 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/836 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/841 [00:00<?, ? examples/s]

In [1]:
from datasets import load_from_disk
tokenized_datasets = load_from_disk('/content/drive/MyDrive/datasets/tokenized_datasets_2')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 84608
    })
    validation: Dataset({
        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 836
    })
    test: Dataset({
        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 841
    })
})

### Обучение

In [2]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

Теперь готовим нашу тонкую настройку. Указываем все необходимые параметры. Записывать все чекпоинты будем к себе на Google диск.

**Внимание!**

Следите за свободным местом на своем диске, если места не будет хватать, можно удалять чекпоинты с худшими метриками.

In [16]:
batch_size = 8
model_name = 'My_Tuned_T5'
model_dir = '/content/drive/MyDrive/CheckPoint'

args = Seq2SeqTrainingArguments(
    output_dir=model_dir,
    eval_strategy="steps",
    eval_steps=500,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",
    save_steps=500,
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    lr_scheduler_type="linear",
    weight_decay=0.1,
    save_total_limit=3,
    max_steps=2000,
    predict_with_generate=True,
    bf16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="none",
    gradient_accumulation_steps=1
)

Далее мы создаем экземпляр DataCollatorForSeq2Seq объекта с помощью токенизатора. Это объект, который формируют батч, используя список элементов набора данных в качестве входных данных и, в некоторых случаях, применяя некоторую обработку. В нашем случае все входные данные и метки в каждом батче будут дополнены до соответствующей им максимальной длины в батче. Заполнение входных данных выполняется с помощью токена [PAD], тогда как заполнение меток выполняется с помощью токена с идентификатором -100 , который является специальным токеном, автоматически игнорируемым функциями потерь в PyTorch.

In [17]:
from transformers import AutoTokenizer

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=256 )

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

Нам осталось определить метрики, по которым будем замерять качество нашей модели. Поскольку у нас задача довольно специфическая, то и метрика будет необычная. Будем использовать метрику **ROUGE**. С тем, что она означает и как вычисляется, можно ознакомиться [здесь](https://en.wikipedia.org/wiki/ROUGE_(metric)).

In [24]:
import numpy as np
import evaluate


metric = evaluate.load("rouge", trust_remote_code=True)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    predictions = predictions.astype(np.int32)
    labels = labels.astype(np.int32)

    predictions = np.where(predictions != -100,
                           predictions,
                           tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions,
                                           skip_special_tokens=True,
                                           clean_up_tokenization_spaces=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels,
                                            skip_special_tokens=True,
                                            clean_up_tokenization_spaces=True)

    # Добавляем отступ для каждой новой строчки
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Вычисляем ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Дополнительно посчитаем ROUGE f1 scores
    result = {key: value * 100 for key, value in result.items()}

    # И посчитаем среднюю длину ответа
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Запускаем обучение!

In [25]:
import torch

In [26]:
def model_init():
    model = AutoModelForSeq2SeqLM.from_pretrained(
    "t5-base",
    torch_dtype=torch.bfloat16 if args.bf16 else torch.float32,
)
    return model

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    # tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [27]:

trainer.train()


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,0.929200,0.718324,0.264400,0.153800,0.249700,0.264400,0.107700
1000,0.690500,0.607277,24.565400,11.615100,22.332800,22.410900,12.602900
1500,0.643400,0.601331,26.602400,12.721900,24.230700,24.348100,12.983300
2000,0.688600,0.600550,26.535800,12.641300,24.197000,24.296200,12.998800


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=2000, training_loss=2.3555975189208986, metrics={'train_runtime': 534.4865, 'train_samples_per_second': 29.935, 'train_steps_per_second': 3.742, 'total_flos': 9743326248960000.0, 'train_loss': 2.3555975189208986, 'epoch': 0.18910741301059})

Что можно сказать про метрику rouge? Она должна расти или падать?

**Максимальный балл - 40**

Загружаем лучший чекпоинт.

In [29]:
trainer.save_model("model")

In [30]:
model = trainer.model

In [ ]:
model_name = # Ваш код здесь
model_dir = # Ваш код здесь

tokenizer = AutoTokenizer.from_pretrained(model_dir)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

### Тестирование

Проверим на каком-нибудь тексте, как работает наша модель. Поскольку нам нужен заголовок, то ограничим его длину от 10 до 64 символов.

In [32]:
text = """
Many financial institutions started building conversational AI, prior to the Covid19
pandemic, as part of a digital transformation initiative. These initial solutions
were high profile, highly personalized virtual assistants — like the Erica chatbot
from Bank of America. As the pandemic hit, the need changed as contact centers were
under increased pressures. As Cathal McGloin of ServisBOT explains in “how it started,
and how it is going,” financial institutions were looking for ways to automate
solutions to help get back to “normal” levels of customer service. This resulted
in a change from the “future of conversational AI” to a real tactical assistant
that can help in customer service. Haritha Dev of Wells Fargo, saw a similar trend.
Banks were originally looking to conversational AI as part of digital transformation
to keep up with the times. However, with the pandemic, it has been more about
customer retention and customer satisfaction. In addition, new use cases came about
as a result of Covid-19 that accelerated adoption of conversational AI. As Vinita
Kumar of Deloitte points out, banks were dealing with an influx of calls about new
concerns, like questions around the Paycheck Protection Program (PPP) loans. This
resulted in an increase in volume, without enough agents to assist customers, and
tipped the scale to incorporate conversational AI. When choosing initial use cases
to support, financial institutions often start with high volume, low complexity
tasks. For example, password resets, checking account balances, or checking the
status of a transaction, as Vinita points out. From there, the use cases can evolve
as the banks get more mature in developing conversational AI, and as the customers
become more engaged with the solutions. Cathal indicates another good way for banks
to start is looking at use cases that are a pain point, and also do not require a
lot of IT support. Some financial institutions may have a multi-year technology
roadmap, which can make it harder to get a new service started. A simple chatbot
for document collection in an onboarding process can result in high engagement,
and a high return on investment. For example, Cathal has a banking customer that
implemented a chatbot to capture a driver’s license to be used in the verification
process of adding an additional user to an account — it has over 85% engagement
with high satisfaction. An interesting use case Haritha discovered involved
educating customers on financial matters. People feel more comfortable asking a
chatbot what might be considered a “dumb” question, as the chatbot is less judgmental.
Users can be more ambiguous with their questions as well, not knowing the right
words to use, as chatbot can help narrow things down.
"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt").to('cuda')
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)

Cathal McGloin of ServisBOT explains “how it started, and how it is going”


### Деплой

Теперь осталось задеплоить нашу замечательную модель.

**Hugging Face Spaces** — это сервис, где вы можете развернуть свои приложения Streamlit или Gradio, чтобы вы могли легко ими поделиться. Он предоставляет бесплатные процессоры и похож на Streamlit Cloud. Однако, когда вы разворачиваете приложение на Hugging Face Spaces, которое использует модель машинного обучения, загруженную на Hugging Face Hub, все могут видеть, что они связаны (т. е. что есть интерактивная демонстрация для этой модели).

Для начала зарегистрируйтесь на Hugging Face. После этого создате токен доступа.

**Внимание!**

При создании токена обязательно выберите токен с правами *WRITE*. Иначе ничего не выйдет.

Затем нам необходимо авторизоваться в нашей учетной записи Hugging Face, поскольку загруженные модели будут привязаны к ней. Вводим сгенерированный токен.

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
https://huggingface.co/Patrik1352/T5Laber

Теперь загружаем нашу модель к себе.

**Максимальный балл - 20**

Готово! У вас теперь есть своя собственная модель. Осталось теперь создать новое пространство в Hugging Face Space.

Для этого вы добавляете новый Space в своем аккаунте (выбирайте только CPU ресурсы, они бесплатные). И обязательно сделайте его **PUBLIC**. Иначе его будет видно только вам.

<img src='https://huggingface.co/blog/assets/29_streamlit-spaces/streamlit.gif' align="center" height=400, width=600>

Далее добавляете 2 файла, приложенные к этому ДЗ.

- Файл **app.py**: Содержит код Streamlit приложения. Это место, где мы загружаем нашу модель из Hugging Face Hub, создаем некоторые интерактивные компоненты и пишем некоторую логику для их соединения.
- Файл **requirements.py**: Содержит все библиотеки Python, используемые в app.py файле (например, transformers библиотеку). Добавлять библиотеку Streamlit в этот файл необязательно.

Вам нужно будет поменять название модели на свое. Однако вы можете с ним поиграться более основательно и добавить новый функционал, но это необязательно. Все готово, переходите на вкладку App и у вас поднимется ваш сервис. В качестве решения просто приложите ссылку на него.

Пример работы сервиса [тут](https://huggingface.co/spaces/AGIvan/title-generation)

In [ ]:
https://huggingface.co/spaces/Patrik1352/T5_Titler

**Максимальный балл - 20**